In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing


In [ ]:

batch_size = 1
num_class = 1000
image_shape = (3, 224, 224)
data_shape = (batch_size,) + image_shape
out_shape = (batch_size, num_class)

# get_workload : 
mod, params = relay.testing.vgg.get_workload(
    num_layers=16, batch_size=batch_size, image_shape=image_shape
    )




opt_level = 3
# opt_level = 1
target = tvm.target.cuda()
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)

# create random input
dev = tvm.cuda()
data = np.random.uniform(-1, 1, size=data_shape).astype('float32')
print("shape of input data :", data.shape)

# create module
module = graph_executor.GraphModule(lib['default'](dev))

# set input and parameter
module.set_input('data', data)

# run
module.run()

# get output
out = module.get_output(0, tvm.nd.empty(out_shape)).asnumpy()

# print first 10 elements of output
print(out.flatten()[0:10])

from tvm.contrib import utils

temp = utils.tempdir()
path_lib = temp.relpath('deploy_lib.tar')
lib.export_library(path_lib)
print(temp.listdir())

# load the module back.
loaded_lib = tvm.runtime.load_module(path_lib)
input_data = tvm.nd.array(data)

module = graph_executor.GraphModule(loaded_lib['default'](dev))
module.run(data=input_data)
out_deploy = module.get_output(0).asnumpy()

graph_json = lib.get_graph_json()
print(graph_json)

# print first 10 elements of output
print(out_deploy.flatten()[0:10])

# check whether the output from deployed module is consistent with original one
tvm.testing.assert_allclose(out_deploy, out, atol=1e-5)


In [17]:
rgraph = tvm.relay.analysis.CallGraph(mod)

In [4]:
import json

In [5]:
ll = json.loads(graph_json)

In [19]:
for i in ll['nodes']:
    print(i)

{'op': 'null', 'name': 'data', 'inputs': []}
{'op': 'null', 'name': 'p0', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_conv2d', 'attrs': {'num_outputs': '1', 'flatten_data': '0', 'data_layout': 'NCHW', 'hash': 'fcf4c3eba41f0b65', 'num_inputs': '2', 'kernel_layout': 'OIHW', 'out_layout': '', 'func_name': 'tvmgen_default_fused_nn_conv2d'}, 'inputs': [[0, 0, 0], [1, 0, 0]]}
{'op': 'null', 'name': 'p1', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_add', 'attrs': {'num_inputs': '2', 'num_outputs': '1', 'hash': '9944b64f2bf8d828', 'flatten_data': '0', 'func_name': 'tvmgen_default_fused_add'}, 'inputs': [[2, 0, 0], [3, 0, 0]]}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_relu', 'attrs': {'num_inputs': '1', 'num_outputs': '1', 'hash': 'ee5a8a7342ec30c9', 'flatten_data': '0', 'func_name': 'tvmgen_default_fused_nn_relu'}, 'inputs': [[4, 0, 0]]}
{'op': 'null', 'name': 'p2', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_contrib_conv2d_winog

In [5]:
out.flatten()[0:10]

array([0.00100052, 0.00099585, 0.00099843, 0.00100801, 0.00099786,
       0.00100267, 0.00100183, 0.00100203, 0.00099908, 0.0010008 ],
      dtype=float32)

In [6]:
# print first 10 elements of output
print(out.flatten()[0:10])

from tvm.contrib import utils

temp = utils.tempdir()
path_lib = temp.relpath('deploy_lib.tar')
lib.export_library(path_lib)
print(temp.listdir())

# load the module back.
loaded_lib = tvm.runtime.load_module(path_lib)
input_data = tvm.nd.array(data)

module = graph_executor.GraphModule(loaded_lib['default'](dev))
module.run(data=input_data)
out_deploy = module.get_output(0).asnumpy()

# print first 10 elements of output
print(out_deploy.flatten()[0:10])

# check whether the output from deployed module is consistent with original one
tvm.testing.assert_allclose(out_deploy, out, atol=1e-5)


[0.00100052 0.00099585 0.00099843 0.00100801 0.00099786 0.00100267
 0.00100183 0.00100203 0.00099908 0.0010008 ]
['deploy_lib.tar']
[0.00100052 0.00099585 0.00099843 0.00100801 0.00099786 0.00100267
 0.00100183 0.00100203 0.00099908 0.0010008 ]
